# CAPSTONE 3. Predicting Next Cryptocurrency Market Cycle Peak
## Data Wrangling

In this notebook we will perform data wrangling for our project. We will:<br>
<ol>1. Retreive historical data for four major cryptocurrencies:<br>
    <ol><i>1.1. Bitcoin (<b>BTC</b>)<br>
        1.2. Ethereum (<b>ETH</b>)<br>
        1.3. XPR (<b>XRP</b>)<br>
        1.4. Litecoin (<b>LTC</b>)<br>
    </ol>
    2. Organize it and make sure it's well defined and ready for the next step - Exploratory Data Analysis
</ol>

In [1]:
#importing all the necessary modules and libraries
import pandas as pd
import os
import glob
from functools import reduce
import datetime as dt

First, let's read all the data we downloaded from YahooFinance.

In [2]:
#creating one dataframe for each token
df_BTC = pd.read_csv('../datasets/BTC-USD.csv', parse_dates=True).sort_values(by='Date', ascending=False)
df_ETH = pd.read_csv('../datasets/ETH-USD.csv', parse_dates=True).sort_values(by='Date', ascending=False)
df_XRP = pd.read_csv('../datasets/XRP-USD.csv', parse_dates=True).sort_values(by='Date', ascending=False)
df_LTC = pd.read_csv('../datasets/LTC-USD.csv', parse_dates=True).sort_values(by='Date', ascending=False)

In [3]:
df_BTC.head(3)

,Date,Open,High,Low,Close,Adj Close,Volume
1998,2021-01-24,31794.328125,32938.765625,31106.685547,31786.878906,31786.878906,4.680768e+10
1997,2021-01-23,NaN,NaN,NaN,NaN,NaN,NaN
1996,2021-01-22,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_BTC.tail(3)

,Date,Open,High,Low,Close,Adj Close,Volume
2,2015-08-08,279.742004,279.928009,260.709991,260.997009,260.997009,58533000.0
1,2015-08-07,278.740997,280.391998,276.365997,279.584991,279.584991,42484800.0
0,2015-08-06,281.906006,281.906006,278.403015,278.576996,278.576996,18792100.0


Now, let's add the token's abbreviation to the columns and remove 'Adj Close' column.

In [5]:
df_BTC = df_BTC.rename({'Open':'Open_BTC', 'High':'High_BTC', 'Low':'Low_BTC', 'Close':'Close_BTC', 'Volume':'Volume_BTC'}, axis=1).drop(columns=['Adj Close'], axis=1)
df_ETH = df_ETH.rename({'Open':'Open_ETH', 'High':'High_ETH', 'Low':'Low_ETH', 'Close':'Close_ETH', 'Volume':'Volume_ETH'}, axis=1).drop(columns=['Adj Close'], axis=1)
df_XRP = df_XRP.rename({'Open':'Open_XRP', 'High':'High_XRP', 'Low':'Low_XRP', 'Close':'Close_XRP', 'Volume':'Volume_XRP'}, axis=1).drop(columns=['Adj Close'], axis=1)
df_LTC = df_LTC.rename({'Open':'Open_LTC', 'High':'High_LTC', 'Low':'Low_LTC', 'Close':'Close_LTC', 'Volume':'Volume_LTC'}, axis=1).drop(columns=['Adj Close'], axis=1)

In [6]:
df_BTC.head(1)

,Date,Open_BTC,High_BTC,Low_BTC,Close_BTC,Volume_BTC
1998,2021-01-24,31794.328125,32938.765625,31106.685547,31786.878906,4.680768e+10


We're still missing one important piece of data - each token's total market capitalization. Let's read more datasets from CoinGecko which have that piece.

In [7]:
df_BTC_cap = pd.read_csv(r'D:\Tutorials\DATASETS\GeckoCryptos\btc-usd-max.csv', parse_dates=True, encoding='utf-8')
df_BTC_cap['snapped_at'] = pd.to_datetime(df_BTC_cap['snapped_at']).dt.date
df_BTC_cap = df_BTC_cap.rename({'snapped_at':'Date', 'market_cap':'MarketCap_BTC'}, axis=1)
df_ETH_cap = pd.read_csv(r'D:\Tutorials\DATASETS\GeckoCryptos\eth-usd-max.csv', parse_dates=True, encoding='utf-8')
df_ETH_cap['snapped_at'] = pd.to_datetime(df_ETH_cap['snapped_at']).dt.date
df_ETH_cap = df_ETH_cap.rename({'snapped_at':'Date', 'market_cap':'MarketCap_ETH'}, axis=1)
df_XRP_cap = pd.read_csv(r'D:\Tutorials\DATASETS\GeckoCryptos\xrp-usd-max.csv', parse_dates=True, encoding='utf-8')
df_XRP_cap['snapped_at'] = pd.to_datetime(df_XRP_cap['snapped_at']).dt.date
df_XRP_cap = df_XRP_cap.rename({'snapped_at':'Date', 'market_cap':'MarketCap_XRP'}, axis=1)
df_LTC_cap = pd.read_csv(r'D:\Tutorials\DATASETS\GeckoCryptos\ltc-usd-max.csv', parse_dates=True, encoding='utf-8')
df_LTC_cap['snapped_at'] = pd.to_datetime(df_LTC_cap['snapped_at']).dt.date
df_LTC_cap = df_LTC_cap.rename({'snapped_at':'Date', 'market_cap':'MarketCap_LTC'}, axis=1)

In [8]:
df_BTC_cap.head(3)

,Date,price,MarketCap_BTC,total_volume
0,2013-04-28,135.30,1.500518e+09,0.0
1,2013-04-29,141.96,1.575032e+09,0.0
2,2013-04-30,135.30,1.501657e+09,0.0


In [9]:
#dropping 'price' and 'total_volume'
df_BTC_cap = df_BTC_cap.drop(columns=['price', 'total_volume'], axis=1)
df_ETH_cap = df_ETH_cap.drop(columns=['price', 'total_volume'], axis=1)
df_XRP_cap = df_XRP_cap.drop(columns=['price', 'total_volume'], axis=1)
df_LTC_cap = df_LTC_cap.drop(columns=['price', 'total_volume'], axis=1)

Great. Now let's merge our caps dataframes into one.

In [12]:
dfs_1_cap = [df_BTC_cap, df_ETH_cap]
df_1_cap = reduce(lambda left,right: pd.merge(left,right, on=['Date'], how='inner'), dfs_1_cap)
dfs_2_cap = [df_1_cap, df_XRP_cap]
df_2_cap = reduce(lambda left,right: pd.merge(left,right, on=['Date'], how='inner'), dfs_2_cap)
dfs_3_cap = [df_2_cap, df_LTC_cap]
df_merged_cap = reduce(lambda left,right: pd.merge(left,right, on=['Date'], how='inner'), dfs_3_cap)
df_merged_cap.head(1)

,Date,MarketCap_BTC,MarketCap_ETH,MarketCap_XRP,MarketCap_LTC
0,2015-08-07,4.015651e+09,0.0,2.596767e+08,1.718568e+08


And let's merge our original dataframes into one.

In [20]:
dfs_1 = [df_BTC, df_ETH]
df_1 = reduce(lambda left,right: pd.merge(left,right, on=['Date'], how='inner'), dfs_1)
dfs_2 = [df_1, df_XRP]
df_2 = reduce(lambda left,right: pd.merge(left,right, on=['Date'], how='inner'), dfs_2)
dfs_3 = [df_2, df_LTC]
df_merged = reduce(lambda left,right: pd.merge(left,right, on=['Date'], how='inner'), dfs_3)
df_merged

,Date,Open_BTC,High_BTC,Low_BTC,Close_BTC,Volume_BTC,Open_ETH,High_ETH,Low_ETH,Close_ETH,...,Open_XRP,High_XRP,Low_XRP,Close_XRP,Volume_XRP,Open_LTC,High_LTC,Low_LTC,Close_LTC,Volume_LTC
0,2021-01-24,31794.328125,32938.765625,31106.685547,31786.878906,4.680768e+10,1329.800537,1362.217529,1225.867188,1348.711914,...,0.274123,0.277297,0.270216,0.272516,2.453091e+09,141.666153,142.619385,134.478394,138.431396,4.809200e+09
1,2021-01-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1993,2015-08-11,264.342010,270.385986,264.093994,270.385986,2.543390e+07,0.708087,1.131410,0.663235,1.067860,...,0.008751,0.008766,0.008591,0.008591,2.824610e+05,3.948740,4.159550,3.942950,4.159550,3.426300e+06
1994,2015-08-10,265.477997,267.032013,262.596008,264.470001,2.097940e+07,0.713989,0.729854,0.636546,0.708448,...,0.008812,0.008905,0.008746,0.008750,4.729730e+05,3.900800,3.980130,3.897610,3.948880,2.239890e+06
1995,2015-08-09,261.115997,267.002991,260.467987,265.083008,2.378960e+07,0.706136,0.879810,0.629191,0.701897,...,0.008484,0.008823,0.008472,0.008808,5.319690e+05,3.843390,3.984260,3.811390,3.898590,3.064680e+06
1996,2015-08-08,279.742004,279.928009,260.709991,260.997009,5.853300e+07,2.793760,2.798810,0.714725,0.753325,...,0.008164,0.008708,0.008164,0.008476,6.782950e+05,4.220990,4.223640,3.835420,3.854750,4.917730e+06


And finally let's add market cap to our merged dataframe.

In [27]:
#converting boths' dataframes 'Date' columns to datetime object for merging
df_merged['Date'] = pd.to_datetime(df_merged['Date']).dt.date
df_merged_cap['Date'] = pd.to_datetime(df_merged_cap['Date']).dt.date

In [35]:
df = df_merged.merge(df_merged_cap, how='outer', on='Date').sort_values(by='Date', ascending=False)

In [36]:
df

,Date,Open_BTC,High_BTC,Low_BTC,Close_BTC,Volume_BTC,Open_ETH,High_ETH,Low_ETH,Close_ETH,...,Volume_XRP,Open_LTC,High_LTC,Low_LTC,Close_LTC,Volume_LTC,MarketCap_BTC,MarketCap_ETH,MarketCap_XRP,MarketCap_LTC
1999,2021-01-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.025179e+11,1.519403e+11,1.223955e+10,9.114576e+09
1998,2021-01-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.005954e+11,1.581672e+11,1.245625e+10,9.404889e+09
0,2021-01-24,31794.328125,32938.765625,31106.685547,31786.878906,4.680768e+10,1329.800537,1362.217529,1225.867188,1348.711914,...,2.453091e+09,141.666153,142.619385,134.478394,138.431396,4.809200e+09,5.967444e+11,1.407732e+11,1.235317e+10,9.138226e+09
1,2021-01-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.132711e+11,1.402719e+11,1.234811e+10,9.151085e+09
2,2021-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.752055e+11,1.307137e+11,1.225147e+10,8.633411e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1993,2015-08-11,264.342010,270.385986,264.093994,270.385986,2.543390e+07,0.708087,1.131410,0.663235,1.067860,...,2.824610e+05,3.948740,4.159550,3.942950,4.159550,3.426300e+06,3.909601e+09,6.453901e+07,2.755990e+08,1.715326e+08
1994,2015-08-10,265.477997,267.032013,262.596008,264.470001,2.097940e+07,0.713989,0.729854,0.636546,0.708448,...,4.729730e+05,3.900800,3.980130,3.897610,3.948880,2.239890e+06,3.817350e+09,4.155631e+07,2.790050e+08,1.635857e+08
1995,2015-08-09,261.115997,267.002991,260.467987,265.083008,2.378960e+07,0.706136,0.879810,0.629191,0.701897,...,5.319690e+05,3.843390,3.984260,3.811390,3.898590,3.064680e+06,NaN,NaN,NaN,NaN
1996,2015-08-08,279.742004,279.928009,260.709991,260.997009,5.853300e+07,2.793760,2.798810,0.714725,0.753325,...,6.782950e+05,4.220990,4.223640,3.835420,3.854750,4.917730e+06,3.745911e+09,8.033948e+07,2.695966e+08,1.607698e+08


Now let's insert each of the token's market cap column after respective token's volume column for better readability.

In [39]:
vol_cols = [col for col in df.columns if 'Volume_' in col]
mc_cols = [col for col in df.columns if 'MarketCap_' in col]
for column in vol_cols:
    print(column, 'index is:', df.columns.get_loc(column))

Volume_BTC index is: 5
Volume_ETH index is: 10
Volume_XRP index is: 15
Volume_LTC index is: 20


In [40]:
col = df.pop("MarketCap_BTC")
df.insert(6, col.name, col)
col = df.pop("MarketCap_ETH")
df.insert(12, col.name, col)
col = df.pop("MarketCap_XRP")
df.insert(18, col.name, col)
col = df.pop("MarketCap_LTC")
df.insert(24, col.name, col)

In [42]:
df.columns

Index(['Date', 'Open_BTC', 'High_BTC', 'Low_BTC', 'Close_BTC', 'Volume_BTC',
       'MarketCap_BTC', 'Open_ETH', 'High_ETH', 'Low_ETH', 'Close_ETH',
       'Volume_ETH', 'MarketCap_ETH', 'Open_XRP', 'High_XRP', 'Low_XRP',
       'Close_XRP', 'Volume_XRP', 'MarketCap_XRP', 'Open_LTC', 'High_LTC',
       'Low_LTC', 'Close_LTC', 'Volume_LTC', 'MarketCap_LTC'],
      dtype='object')

Great. Now all columns are in the right place.

In [37]:
#looking how many observations and features we have
df.shape

(2000, 25)

We have 2000 observations and 25 features. Above we noticed we had some missing data.

In [45]:
df.isna().sum()

Date             0
Open_BTC         6
High_BTC         6
Low_BTC          6
Close_BTC        6
Volume_BTC       6
MarketCap_BTC    2
Open_ETH         6
High_ETH         6
Low_ETH          6
Close_ETH        6
Volume_ETH       6
MarketCap_ETH    2
Open_XRP         6
High_XRP         6
Low_XRP          6
Close_XRP        6
Volume_XRP       6
MarketCap_XRP    1
Open_LTC         6
High_LTC         6
Low_LTC          6
Close_LTC        6
Volume_LTC       6
MarketCap_LTC    2
dtype: int64

We don't have a lot of missing values so we wil just drop them.

In [46]:
df.dropna(axis=0, inplace=True)
df.isnull().any()

Date             False
Open_BTC         False
High_BTC         False
Low_BTC          False
Close_BTC        False
Volume_BTC       False
MarketCap_BTC    False
Open_ETH         False
High_ETH         False
Low_ETH          False
Close_ETH        False
Volume_ETH       False
MarketCap_ETH    False
Open_XRP         False
High_XRP         False
Low_XRP          False
Close_XRP        False
Volume_XRP       False
MarketCap_XRP    False
Open_LTC         False
High_LTC         False
Low_LTC          False
Close_LTC        False
Volume_LTC       False
MarketCap_LTC    False
dtype: bool

Great. No more missing values. Let's take a look at our data shape once again.

In [47]:
df.shape

(1992, 25)

We only dropped 8 observatios. Do we have any duplicates?

In [48]:
df.duplicated().any()

False

No duplicates. Our data is ready for the next strep - Exploratory Data Analysis.

In [ ]:
#saving the data
datapath = 'D://Tutorials/SDST/My Projects/Capstone3/DW'
if not os.path.exists(datapath):
    os.mkdir(datapath)
datapath_DW = os.path.join(datapath, 'Data_for_EDA.csv')
if not os.path.exists(datapath_DW):
    df.to_csv(datapath_DW, index=False)